In [21]:
import pandas as pd
import json
import numpy as np
import os
import torch 
import cv2 
import matplotlib.pyplot as plt
ann_dir = '../annotation/annotations/panoptic_val2017.json'
root_dir = '../data/val2017/'


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [3]:
with open(ann_dir, 'r') as f:
    temp = json.load(f)

In [8]:
def get_items(ann_dir):
    with open(ann_dir, 'r') as f:
        temp = json.load(f)
    image_list = []
    ctg_df = pd.DataFrame(temp['categories'])
    id2ctg = dict(ctg_df.set_index('id')['name'])
    ctg2id = dict(ctg_df.set_index('name')['id'])
    for a in temp['annotations']:
        image_id = a['file_name'][:-4]
        
        bbox = np.stack([i['bbox'] for i in a['segments_info']])
        labels = np.asarray([i['category_id'] for i in a['segments_info']])
        image_list.append({'image_id': image_id,
                           'bbox': bbox,
                           'labels': labels})
    return np.asanyarray(image_list), id2ctg

In [ ]:
def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

In [214]:
image_list, id2dtg = get_items(ann_dir)
img_id,bbox,_ = image_list[4002].values()
img = cv2.imread(f'{root_dir}{img_id}.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
# ss.setBaseImage(img)
# ss.switchToSelectiveSearchFast()
# ssresults = ss.process()


In [138]:
boxes = bbox
boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
boxes[:, 3] = boxes[:, 1] + boxes[:, 3]

for idx, result in enumerate(ssresults):
    if idx < 2000:
        for i, gtval in enumerate(boxes):    # ground truth
            x, y, w, h = result
            box2 = [x,y,x+w,y+h]
            iou = IoU(gtval, box2)
            if iou > 0.7:
                print(box2)
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255))


[0, 0, 612, 612]
[0, 0, 612, 513]
[0, 173, 612, 612]
[0, 110, 436, 612]
[0, 0, 528, 612]
[34, 0, 612, 598]
[0, 0, 510, 612]
[34, 0, 464, 598]
[0, 0, 453, 612]
[0, 113, 612, 612]
[49, 0, 612, 612]
[0, 0, 612, 477]
[0, 0, 595, 517]
[0, 114, 612, 612]
[61, 0, 612, 612]
[0, 0, 612, 519]
[0, 0, 612, 517]
[0, 0, 612, 470]


In [174]:
for box, label in zip(bbox, labels):
    x,y,w,h = box
    cv2.rectangle(img,(x,y),(x+w, y+h),(0,0,255),2)
    cv2.putText(img, f'{label}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)

In [164]:
labels = [id2dtg[i] for i in _]